# Kiva

## Situation

You work as a Data Scientist at the P-2-P platform https://www.kiva.org/, which was founded a year ago. Now you are looking to expand your business. Your team has split up and each analyst has a subset of the data. Your task is to find out insights for your platform in an exploratory data analysis.

Requirements:

- complete data analysis
- documentation of the work steps
- comprehensible and meaningful explanations about approach/decisions
- for the evaluation: name 3 different plots (plot functions)
- customizing of the plots
- Analysis of the plots
- Coding


## CRISP-DM - Business Understanding

Your business model is to operate a platform (crowd-investing) where people who have a business idea, but not the money needed, can register and raise money for their project within a given time. 

On the other hand, you have funders who would like to invest their money in projects and are looking for investments. 

So as an intermediary, your platform brings borrowers and funders together. 

Your database is the history of your platform. 

**Expansion Business Model**

All projects are completed projects, i.e. the time to raise money for your project has expired. 
Your business model is to pay out the collected money even if the target amount is not reached.

You earn your money with a commission for each project that lands on your platform. 

The backer gets an interest for the money loan.

## CRISP-DM - Data Understanding

- funded_amount - Amount received/paid out at the end of the crowding period in USD.
- loan_amount - target amount (amount you wanted to reach for the project) in USD
- activity - subcategory to which the goal of the crowd project belongs thematically
- sector - upper category to which the crowdproject's topic belongs
- use - short description of what the money will be used for
- country_code - country code according to ISO standard
- country - country name according to ISO standard
- region - region
- currency - currency in which the funded_amount was then paid out
- term in months - duration over which the loan is to be paid
- lender_count - lender (how many people have given money for the project)
- borrower_genders - gender and number of borrowers, i.e. those who initiated the crowdproject       
- repayment interval - contractually agreed repayment modalities/rhythm

## CRISP-DM - Data Preparation

### Reading in data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
df = pd.read_csv(
    'data_abschlussprojekt.csv',
    sep='#',                      # Reading the .csv files with ',' fails. Upon investigation the seperator used is '#'
    index_col=0                   # An index column is provided in the .csv dataset, use this instead of generating a new one
)

### Initial data checks
#### Column names

In [3]:
# All column names have a space at the beginning which is not obvious without expection
# To improve usability of the dataset for the rest of the analysis, the spaces will be removed

df.columns

Index([' funded_amount', ' loan_amount', ' activity', ' sector', ' use',
       ' country_code', ' country', ' region', ' currency', ' term_in_months',
       ' lender_count', ' borrower_genders', ' repayment_interval'],
      dtype='object')

In [4]:
df.columns = df.columns.str.strip()
df.columns

Index(['funded_amount', 'loan_amount', 'activity', 'sector', 'use',
       'country_code', 'country', 'region', 'currency', 'term_in_months',
       'lender_count', 'borrower_genders', 'repayment_interval'],
      dtype='object')

#### Column data types

In [5]:
# Checking to ensure that columns have the appropriate data types. 
# All numeric columns have appropriate data types. 
df.dtypes

funded_amount         float64
loan_amount           float64
activity               object
sector                 object
use                    object
country_code           object
country                object
region                 object
currency               object
term_in_months        float64
lender_count            int64
borrower_genders       object
repayment_interval     object
dtype: object

#### System known missing values

In [6]:
# There are 671,205 total entries, compared to the non-null numbers below shows the initial system detected missing values
# per column. Columns with known missing values: use, country_code, region, borrower_genders
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 671205 entries, 0 to 671204
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   funded_amount       671205 non-null  float64
 1   loan_amount         671205 non-null  float64
 2   activity            671205 non-null  object 
 3   sector              671205 non-null  object 
 4   use                 666973 non-null  object 
 5   country_code        671197 non-null  object 
 6   country             671205 non-null  object 
 7   region              614405 non-null  object 
 8   currency            671205 non-null  object 
 9   term_in_months      671205 non-null  float64
 10  lender_count        671205 non-null  int64  
 11  borrower_genders    666984 non-null  object 
 12  repayment_interval  671205 non-null  object 
dtypes: float64(3), int64(1), object(9)
memory usage: 71.7+ MB


### Data quality

#### Removing leading and trailing spaces from data

Before identifying duplicates, it is important to ensure that data in string columns do not have spaces at the start or end in order to avoid duplicates being missed.

In [7]:
# Extracting which columns have string datatypes, as the .str.strip() cannot be applied to columns
# that do not have string as a datatype, such as int or float columns.
string_columns = df.dtypes.loc[df.dtypes=='object'].index

# Programatically looping over columns rather than copy and pasting each column name 
for column in string_columns:
    df[column] = df[column].str.strip()

#### Capitalisation of data

Before identifying duplicates, it is also important to ensure that data in string columns are capitalised in a consistent manner since Python is a case sensitive language. Otherwise, two values that are informationally the same will not be identified as the same by Python.

The column 'use' will not be checked, as this is a free-text field and will not be used as a field to filter on later.

By sorting values and printing unique values, any lowercase cases will be sorted to the end of the returned list.

In [8]:
df.loc[:,'activity'].sort_values().unique()

array(['Adult Care', 'Agriculture', 'Air Conditioning', 'Animal Sales',
       'Aquaculture', 'Arts', 'Auto Repair', 'Bakery', 'Balut-Making',
       'Barber Shop', 'Beauty Salon', 'Beekeeping', 'Beverages',
       'Bicycle Repair', 'Bicycle Sales', 'Blacksmith', 'Bookbinding',
       'Bookstore', 'Bricks', 'Butcher Shop', 'Cafe', 'Call Center',
       'Carpentry', 'Catering', 'Cattle', 'Celebrations', 'Cement',
       'Cereals', 'Charcoal Sales', 'Cheese Making', 'Child Care',
       'Cleaning Services', 'Cloth & Dressmaking Supplies', 'Clothing',
       'Clothing Sales', 'Cobbler', 'Communications', 'Computer',
       'Computers', 'Construction', 'Construction Supplies',
       'Consumer Goods', 'Cosmetics Sales', 'Crafts', 'Dairy',
       'Decorations Sales', 'Dental', 'Education provider',
       'Electrical Goods', 'Electrician', 'Electronics Repair',
       'Electronics Sales', 'Embroidery', 'Energy', 'Entertainment',
       'Event Planning', 'Farm Supplies', 'Farming', 'Film',
 

In [9]:
df.loc[:,'sector'].sort_values().unique()

array(['Agriculture', 'Arts', 'Clothing', 'Construction', 'Education',
       'Entertainment', 'Food', 'Health', 'Housing', 'Manufacturing',
       'Personal Use', 'Retail', 'Services', 'Transportation',
       'Wholesale'], dtype=object)

In [10]:
df.loc[:,'country_code'].sort_values().unique()

array(['AF', 'AL', 'AM', 'AZ', 'BF', 'BI', 'BJ', 'BO', 'BR', 'BT', 'BZ',
       'CD', 'CG', 'CI', 'CL', 'CM', 'CN', 'CO', 'CR', 'DO', 'EC', 'EG',
       'GE', 'GH', 'GT', 'GU', 'HN', 'HT', 'ID', 'IL', 'IN', 'IQ', 'JO',
       'KE', 'KG', 'KH', 'LA', 'LB', 'LR', 'LS', 'MD', 'MG', 'ML', 'MM',
       'MN', 'MR', 'MW', 'MX', 'MZ', 'NG', 'NI', 'NP', 'PA', 'PE', 'PH',
       'PK', 'PR', 'PS', 'PY', 'RW', 'SB', 'SL', 'SN', 'SO', 'SR', 'SS',
       'SV', 'TG', 'TH', 'TJ', 'TL', 'TR', 'TZ', 'UA', 'UG', 'US', 'VC',
       'VI', 'VN', 'VU', 'WS', 'XK', 'YE', 'ZA', 'ZM', 'ZW', nan],
      dtype=object)

In [11]:
df.loc[:,'country'].sort_values().unique()

array(['Afghanistan', 'Albania', 'Armenia', 'Azerbaijan', 'Belize',
       'Benin', 'Bhutan', 'Bolivia', 'Brazil', 'Burkina Faso', 'Burundi',
       'Cambodia', 'Cameroon', 'Chile', 'China', 'Colombia', 'Congo',
       'Costa Rica', "Cote D'Ivoire", 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Georgia', 'Ghana', 'Guam', 'Guatemala',
       'Haiti', 'Honduras', 'India', 'Indonesia', 'Iraq', 'Israel',
       'Jordan', 'Kenya', 'Kosovo', 'Kyrgyzstan',
       "Lao People's Democratic Republic", 'Lebanon', 'Lesotho',
       'Liberia', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mexico',
       'Moldova', 'Mongolia', 'Mozambique', 'Myanmar (Burma)', 'Namibia',
       'Nepal', 'Nicaragua', 'Nigeria', 'Pakistan', 'Palestine', 'Panama',
       'Paraguay', 'Peru', 'Philippines', 'Puerto Rico', 'Rwanda',
       'Saint Vincent and the Grenadines', 'Samoa', 'Senegal',
       'Sierra Leone', 'Solomon Islands', 'Somalia', 'South Africa',
       'South Sudan', 'Suriname', 'Tajik

In [12]:
df.loc[:,'currency'].sort_values().unique()

array(['ALL', 'AMD', 'AZN', 'BIF', 'BOB', 'BRL', 'BZD', 'CLP', 'CNY',
       'COP', 'CRC', 'DOP', 'EGP', 'EUR', 'GEL', 'GHS', 'GTQ', 'HNL',
       'HTG', 'IDR', 'ILS', 'INR', 'JOD', 'KES', 'KGS', 'KHR', 'LAK',
       'LBP', 'LRD', 'LSL', 'MDL', 'MGA', 'MMK', 'MNT', 'MWK', 'MXN',
       'MZN', 'NAD', 'NGN', 'NIO', 'NPR', 'PEN', 'PHP', 'PKR', 'PYG',
       'RWF', 'SBD', 'SLL', 'SRD', 'SSP', 'THB', 'TJS', 'TRY', 'TZS',
       'UAH', 'UGX', 'USD', 'VND', 'VUV', 'WST', 'XAF', 'XCD', 'XOF',
       'YER', 'ZAR', 'ZMW', 'ZWD'], dtype=object)

In [13]:
# Although there are lowercase entries here, it is not an issue as all values are lowercase.
df.loc[:,'repayment_interval'].sort_values().unique()

array(['bullet', 'irregular', 'monthly', 'weekly'], dtype=object)

In [14]:
# For the column region, unique provides too many vaules to check by eye.
# However, we see from the output below that at least one value is not capitalised
df.loc[:,'region'].unique()

array(['Lahore', 'Maynaguri', 'Abdul Hakeem', ..., 'Gbenikoro Village',
       'Morimaraia', 'alejandria'], dtype=object)

In [15]:
# There are 6120 cases of regions that are not captilized, with 286 unique cases.
# .notnull is required as otherwise an error is returned for trying to perform a string methond on a np.nan.
lower_case_region = df.loc[(df.region.str[0].str.islower()) & (df.region.notnull()), 'region']
lower_case_region

87        jenin
126        gaza
279        gaza
312        gaza
313       jenin
          ...  
670878    turbo
670978    turbo
670991    turbo
671037    turbo
671087    turbo
Name: region, Length: 6120, dtype: object

In [16]:
lower_case_region.value_counts()

baguida                                828
karatina                               563
hodeidah                               467
nyeri                                  380
sindangan, zamboanga del norte         321
                                      ... 
tuburan, aloran, mis.occ.                1
p-2, culpan, aloran. mis.occ.            1
shichirai-kakamega                       1
p-3, eastern poblacion, lopez jaena      1
alejandria                               1
Name: region, Length: 286, dtype: int64

In [17]:
df['region'] = df['region'].str.capitalize()

#### Business Logic
Per the business understanding, it would be expected that the column funded_amount should not exceed the value in loan_amount, as then the borrower would be receiving more money than they had requested. We check this to ensure there is not a business bug. 

In [18]:
df.loc[(df.funded_amount > df.loan_amount)]

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
277188,425.0,400.0,General Store,Retail,"to buy beverages, rice, laundry detergent, sug...",MZ,Mozambique,"Boane, maputo",MZN,17.0,11,male,monthly
338159,3400.0,3000.0,Farm Supplies,Agriculture,"to pay for wires for the grape orchard, cover ...",AM,Armenia,"Hoktember village, armavir region",USD,38.0,84,male,monthly


There are two cases where borrowers received more funding than they requested. These two cases will be passed along to the product team to investigate the business bug and customer relationship teams to investigate further and to reach out the corresponding borrowers if necessary. 

For this analysis, we will lower the funded amount to the loan amount to reflect a 100% funded status as borrowers did receive at least the amount they requested.

In [19]:
business_bug_index = df.loc[(df.funded_amount > df.loan_amount)].index
df.loc[(df.funded_amount > df.loan_amount), 'funded_amount'] = df.loc[(df.funded_amount > df.loan_amount), 'loan_amount']
df.loc[df.index.isin(business_bug_index)]

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
277188,400.0,400.0,General Store,Retail,"to buy beverages, rice, laundry detergent, sug...",MZ,Mozambique,"Boane, maputo",MZN,17.0,11,male,monthly
338159,3000.0,3000.0,Farm Supplies,Agriculture,"to pay for wires for the grape orchard, cover ...",AM,Armenia,"Hoktember village, armavir region",USD,38.0,84,male,monthly


### Check for duplicates

#### Definition of a duplicate

Since we do not have a unique identifier per crowd-investment, and all values in all columns cannot be expected to be unique, combinations of columns together need to be used in an attempt to identify duplicate entries. The most confident we can be that duplicate exists is if two entries are exactly the same across all columns. The likelihood of any two rows having exactly the same values and being valid is low enough to use this as a criteria, especially since the 'use' column is a free-text field.

#### Identifying duplicates

In [20]:
duplicates = df.loc[df.duplicated()==True].index
duplicates

Int64Index([   498,    606,    808,   1703,   2317,   2606,   3266,   3440,
              3602,   3605,
            ...
            671195, 671196, 671197, 671198, 671199, 671200, 671201, 671202,
            671203, 671204],
           dtype='int64', length=25580)

#### Handling Duplicates

Since all values across dupliate rows are the same, we do not need to be consider which of the duplicate rows to remove as long as one occurance remains. Afterwards we reset the index. 

In [21]:
df_duplicate_dropped = df.drop(index=duplicates)
df_duplicate_dropped.reset_index(drop=True, inplace=True)

### Missing values

#### Identify and convert missing value synonyms

First, we eyeball the data to see if there are any obvious synonyms

In [22]:
df_duplicate_dropped.loc[:,'funded_amount'].unique()

array([3.0000e+02, 5.7500e+02, 1.5000e+02, 2.0000e+02, 4.0000e+02,
       2.5000e+02, 4.7500e+02, 6.2500e+02, 2.2500e+02, 8.7500e+02,
       3.5000e+02, 1.2500e+02, 4.5000e+02, 2.2250e+03, 6.0000e+02,
       3.1750e+03, 1.7500e+02, 5.5000e+02, 7.0000e+02, 8.0000e+02,
       4.2750e+03, 1.5000e+03, 3.2500e+02, 2.7500e+02, 1.0750e+03,
       2.0000e+03, 2.7750e+03, 1.5500e+03, 1.0000e+03, 3.0000e+03,
       3.0500e+03, 5.4750e+03, 5.7750e+03, 2.2000e+03, 5.0000e+03,
       1.2500e+03, 3.9750e+03, 9.2500e+02, 1.9250e+03, 4.0000e+03,
       1.1500e+03, 1.3000e+03, 4.6250e+03, 8.5000e+02, 7.5000e+02,
       2.9750e+03, 1.1750e+03, 4.7500e+03, 3.7500e+02, 1.0250e+03,
       1.2000e+03, 1.3250e+03, 7.2500e+02, 2.1000e+03, 3.4000e+03,
       2.9000e+03, 4.2500e+02, 2.8500e+03, 4.2500e+03, 5.0000e+02,
       1.2250e+03, 2.6250e+03, 2.7500e+03, 1.6250e+03, 1.4750e+03,
       2.4000e+03, 2.1250e+03, 4.1750e+03, 2.5000e+03, 1.7000e+03,
       1.0000e+04, 5.1000e+03, 6.7500e+02, 5.2500e+02, 3.8000e

In [23]:
df_duplicate_dropped.loc[:,'loan_amount'].unique()

array([3.0000e+02, 5.7500e+02, 1.5000e+02, 2.0000e+02, 4.0000e+02,
       2.5000e+02, 4.7500e+02, 6.2500e+02, 2.2500e+02, 8.7500e+02,
       3.5000e+02, 1.2500e+02, 4.5000e+02, 2.2250e+03, 6.0000e+02,
       3.1750e+03, 1.7500e+02, 5.5000e+02, 7.0000e+02, 8.0000e+02,
       4.2750e+03, 1.5000e+03, 3.2500e+02, 2.7500e+02, 1.0750e+03,
       2.0000e+03, 2.7750e+03, 1.5500e+03, 1.0000e+03, 3.0000e+03,
       3.0500e+03, 5.0000e+03, 5.4750e+03, 5.7750e+03, 2.2000e+03,
       1.2500e+03, 3.9750e+03, 9.2500e+02, 2.4000e+03, 4.0000e+03,
       1.1500e+03, 1.3000e+03, 4.6250e+03, 8.5000e+02, 7.5000e+02,
       2.9750e+03, 1.1750e+03, 4.7500e+03, 3.7500e+02, 1.0250e+03,
       1.2000e+03, 1.3250e+03, 7.2500e+02, 2.1000e+03, 3.4000e+03,
       2.9000e+03, 4.2500e+02, 2.8500e+03, 4.2500e+03, 5.0000e+02,
       1.2250e+03, 2.7500e+03, 1.6250e+03, 1.4750e+03, 2.1250e+03,
       4.1750e+03, 2.5000e+03, 1.7000e+03, 1.0000e+04, 5.1000e+03,
       6.7500e+02, 5.2500e+02, 3.8000e+03, 4.5000e+03, 6.5000e

In [24]:
df_duplicate_dropped.loc[:,'activity'].unique()

array(['Fruits & Vegetables', 'Rickshaw', 'Transportation', 'Embroidery',
       'Milk Sales', 'Services', 'Dairy', 'Beauty Salon', 'Manufacturing',
       'Food Production/Sales', 'Wholesale', 'General Store',
       'Clothing Sales', 'Poultry', 'Tailoring', 'Sewing', 'Bakery',
       'Restaurant', 'Food Stall', 'Farming', 'Construction Supplies',
       'Personal Products Sales', 'Home Products Sales',
       'Natural Medicines', 'Fish Selling', 'Education provider',
       'Shoe Sales', 'Machinery Rental', 'Butcher Shop', 'Pigs',
       'Personal Expenses', 'Food Market', 'Cosmetics Sales',
       'Personal Housing Expenses', 'Retail', 'Energy', 'Grocery Store',
       'Construction', 'Agriculture', 'Motorcycle Transport',
       'Charcoal Sales', 'Food', 'Pharmacy', 'Fishing', 'Timber Sales',
       'Cattle', 'Electronics Repair', 'Electronics Sales', 'Vehicle',
       'Cafe', 'Blacksmith', 'Higher education costs', 'Used Clothing',
       'Fuel/Firewood', 'Upholstery', 'Catering',

In [25]:
df_duplicate_dropped.loc[:,'sector'].unique()

array(['Food', 'Transportation', 'Arts', 'Services', 'Agriculture',
       'Manufacturing', 'Wholesale', 'Retail', 'Clothing', 'Construction',
       'Health', 'Education', 'Personal Use', 'Housing', 'Entertainment'],
      dtype=object)

In [26]:
df_duplicate_dropped.loc[:,'use'].unique()

array(['To buy seasonal, fresh fruits to sell.',
       'to repair and maintain the auto rickshaw used in their business.',
       'To repair their old cycle-van and buy another one to rent out as a source of income',
       ...,
       'Pretend the issue with loan got addressed by Kiva Coordinator.',
       'Kiva Coordinator replaced loan use. Should see this in viewdiff.',
       'Edited loan use in english.'], dtype=object)

In [27]:
df_duplicate_dropped.loc[:,'country_code'].unique()

array(['PK', 'IN', 'KE', 'NI', 'SV', 'TZ', 'PH', 'PE', 'SN', 'KH', 'LR',
       'VN', 'IQ', 'HN', 'PS', 'MN', 'US', 'ML', 'CO', 'TJ', 'GT', 'EC',
       'BO', 'YE', 'GH', 'SL', 'HT', 'CL', 'JO', 'UG', 'BI', 'BF', 'TL',
       'ID', 'GE', 'UA', 'XK', 'AL', 'CD', 'CR', 'SO', 'ZW', 'CM', 'TR',
       'AZ', 'DO', 'BR', 'MX', 'KG', 'AM', 'PY', 'LB', 'WS', 'IL', 'RW',
       'ZM', 'NP', 'CG', 'MZ', 'ZA', 'TG', 'BJ', 'BZ', 'SR', 'TH', 'NG',
       'MR', 'VU', 'PA', 'VI', 'VC', 'LA', 'MW', 'MM', 'MD', 'SS', 'SB',
       'CN', 'EG', 'GU', 'AF', 'MG', nan, 'PR', 'LS', 'CI', 'BT'],
      dtype=object)

In [28]:
df_duplicate_dropped.loc[:,'country'].unique()

array(['Pakistan', 'India', 'Kenya', 'Nicaragua', 'El Salvador',
       'Tanzania', 'Philippines', 'Peru', 'Senegal', 'Cambodia',
       'Liberia', 'Vietnam', 'Iraq', 'Honduras', 'Palestine', 'Mongolia',
       'United States', 'Mali', 'Colombia', 'Tajikistan', 'Guatemala',
       'Ecuador', 'Bolivia', 'Yemen', 'Ghana', 'Sierra Leone', 'Haiti',
       'Chile', 'Jordan', 'Uganda', 'Burundi', 'Burkina Faso',
       'Timor-Leste', 'Indonesia', 'Georgia', 'Ukraine', 'Kosovo',
       'Albania', 'The Democratic Republic of the Congo', 'Costa Rica',
       'Somalia', 'Zimbabwe', 'Cameroon', 'Turkey', 'Azerbaijan',
       'Dominican Republic', 'Brazil', 'Mexico', 'Kyrgyzstan', 'Armenia',
       'Paraguay', 'Lebanon', 'Samoa', 'Israel', 'Rwanda', 'Zambia',
       'Nepal', 'Congo', 'Mozambique', 'South Africa', 'Togo', 'Benin',
       'Belize', 'Suriname', 'Thailand', 'Nigeria', 'Mauritania',
       'Vanuatu', 'Panama', 'Virgin Islands',
       'Saint Vincent and the Grenadines',
       "Lao Peo

In [29]:
df_duplicate_dropped.loc[:,'region'].unique()

array(['Lahore', 'Maynaguri', 'Abdul hakeem', ...,
       'Chililique-san ignacio', 'Gbenikoro village', 'Morimaraia'],
      dtype=object)

In [30]:
df_duplicate_dropped.loc[:,'currency'].unique()

array(['PKR', 'INR', 'KES', 'NIO', 'USD', 'TZS', 'PHP', 'PEN', 'XOF',
       'LRD', 'VND', 'HNL', 'MNT', 'COP', 'GTQ', 'TJS', 'BOB', 'YER',
       'KHR', 'GHS', 'SLL', 'HTG', 'CLP', 'JOD', 'UGX', 'BIF', 'IDR',
       'GEL', 'UAH', 'EUR', 'ALL', 'CRC', 'XAF', 'TRY', 'AZN', 'DOP',
       'BRL', 'MXN', 'KGS', 'AMD', 'PYG', 'LBP', 'WST', 'ILS', 'RWF',
       'ZMW', 'NPR', 'MZN', 'ZAR', 'BZD', 'SRD', 'NGN', 'VUV', 'XCD',
       'MWK', 'LAK', 'MMK', 'ZWD', 'MDL', 'SSP', 'SBD', 'CNY', 'EGP',
       'MGA', 'NAD', 'LSL', 'THB'], dtype=object)

In [31]:
# All values are ints although data type is float. Convert to int later to save memory.
df_duplicate_dropped.loc[:,'term_in_months'].unique()

array([ 12.,  11.,  43.,  14.,   4.,  13.,  10.,   8.,   5.,  20.,   7.,
         3.,  17.,  15.,   9.,  39.,  23.,  26.,   6.,  30.,  22.,  25.,
        27.,  16.,  52.,  18.,  36.,  19.,  28.,  62.,  32.,  24.,  21.,
        50.,  38.,   2.,  35.,  72., 137.,   1.,  49.,  33.,  42.,  29.,
        37.,  51., 113.,  79.,  31.,  44.,  74.,  34.,  48., 114.,  40.,
       124., 104.,  63.,  85.,  78.,  70.,  86.,  61.,  60.,  67.,  55.,
        53.,  41.,  68., 143.,  77., 130.,  45., 111., 134., 107., 142.,
       148.,  56., 122., 133., 141., 110.,  81., 106.,  54., 147., 112.,
        59., 145., 121., 109.,  80.,  47.,  97.,  75., 101., 128.,  98.,
        87.,  71.,  66.,  46., 125.,  76.,  73., 120., 144., 118., 131.,
        65., 108.,  58., 123.,  84.,  99.,  82.,  92.,  69.,  91.,  57.,
        90.,  93., 129.,  89.,  88.,  64., 126., 138., 158.,  83., 100.,
       105., 132.,  96., 127., 135.,  95., 154., 156.,  94., 115., 102.,
       116., 136., 103., 139., 146.])

In [32]:
df_duplicate_dropped.loc[:,'lender_count'].unique()

array([  12,   14,    6,    8,   16,   19,   24,    3,   10,    7,    9,
         11,   25,    4,   18,   13,   20,   17,    5,   58,   15,   93,
          2,    1,  114,   32,  144,   26,   41,   71,  101,   21,   46,
         34,   62,   98,   97,   92,  169,   40,   53,  154,   77,   22,
        261,   35,   84,   38,   31,  152,   54,   37,  110,   43,  162,
         28,   44,   39,   27,   50,   59,   60,   89,  116,   36,   49,
         72,   23,   96,   86,  111,   52,   83,   51,  151,   63,  147,
        280,  106,  171,  135,  161,  108,   48,   29,   47,   80,  173,
         55,   45,   67,  102,   66,  134,   56,   75,   69,  158,   33,
         30,  164,   65,  131,  273,   42,  166,  138,   82,   57,  107,
         68,  174,  126,   88,   79,   90,  100,  130,   64,   70,   85,
         94,  149,  150,  153,  163,   73,   74,  133,  120,   76,   87,
        141,  200,   61,  132,  122,  245,  168,   81,  109,  112,  228,
        105,  140,  117,  185,  104,   95,  103,  1

In [33]:
# An indepth way to check whether there is an error in data here would be to take each entry, split values and 
# check whether it does not match female or male. 
df_duplicate_dropped.loc[:,'borrower_genders'].sort_values().unique()

array(['female', 'female, female', 'female, female, female', ...,
       'male, male, male, male, male, male, male, male, male, male, male, male, male, male, male, male, male, male, male, male',
       'male, male, male, male, male, male, male, male, male, male, male, male, male, male, male, male, male, male, male, male, male, male',
       nan], dtype=object)

In [34]:
df_duplicate_dropped.loc[:,'repayment_interval'].unique()

array(['irregular', 'bullet', 'monthly', 'weekly'], dtype=object)

#### Total missing values

Having not identified any synonyms for missing values, we can proceed with the predefined np.nan data.

In [35]:
df_duplicate_dropped.isnull().mean()*100

funded_amount         0.000000
loan_amount           0.000000
activity              0.000000
sector                0.000000
use                   0.603911
country_code          0.001239
country               0.000000
region                8.694831
currency              0.000000
term_in_months        0.000000
lender_count          0.000000
borrower_genders      0.602207
repayment_interval    0.000000
dtype: float64

#### How to handle the missing values

**Column: use**

Free text field. We cannot directly use this in an EDA or map on a graph. It would require an additional TextMining analysis that is outside of the scope for this project. As such I will drop the column. Sector/Activity will capture the larger picture of what is included in use while allowing us to conduct an EDA and plot data.

**Column: country_code**

Missing values from country_code can be filled by using information from country, since country is not missing any vaues, and the country's corresponding ISO code.

**Column: region**

8.7% of all loans are missing a region entry. As this is categorical data, we will replace the missing values with the most frequent region for each country.

In [36]:
df_duplicate_dropped.loc[df_duplicate_dropped.country=="Bhutan",'region'].value_counts()

Thimphu    1
Name: region, dtype: int64

However some countries do not have any entries with a region (Guam is an example, below). In this case we will use the country name as the region, as we cannot decern a finer granularity. 

In [37]:
df_duplicate_dropped.loc[df_duplicate_dropped.country=="Guam",'region'].value_counts()

Series([], Name: region, dtype: int64)

**Column: borrower_genders**

Option 1: Take the mode of the group make up. Disadvantage is that the data is actually numerical, but currently can't be used as such because of the data structure used. This results in a loss of information, as we only take the most frequent result and ignore all other information available in the dataset (count of borrowers, count by gender). Better would be to include this information by use of mean or median. 

Option 2: Convert the data into a count of people per case. Doing this only for the total number of people in a group would lose the available information of gender. To retain all of the information available, I will create three columns:
- male_borrower_count
- female_borrower_count
- borrower_count

In [38]:
# It would be better to use gender as an argument input however with the time available I was unable to make this 
# work while calling the function with .apply().
def borrower_count_male(x):
    counter = Counter(x.split(', '))
    return counter['male']

df_duplicate_dropped['male_borrower_count'] = df_duplicate_dropped.loc[df_duplicate_dropped['borrower_genders'].notnull(),'borrower_genders'].apply(func=borrower_count_male)
df_duplicate_dropped.head(2)

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval,male_borrower_count
0,300.0,300.0,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,12.0,12,female,irregular,0.0
1,575.0,575.0,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,11.0,14,"female, female",irregular,0.0


In [39]:
def borrower_count_female(x):
    counter = Counter(x.split(', '))
    return counter['female']

df_duplicate_dropped['female_borrower_count'] = df_duplicate_dropped.loc[df_duplicate_dropped['borrower_genders'].notnull(),'borrower_genders'].apply(func=borrower_count_female)
df_duplicate_dropped.head(2)

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval,male_borrower_count,female_borrower_count
0,300.0,300.0,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,12.0,12,female,irregular,0.0,1.0
1,575.0,575.0,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,11.0,14,"female, female",irregular,0.0,2.0


#### Handle the missing values
**Drop column 'use'**

In [40]:
df_fix_missing_values = df_duplicate_dropped.drop('use', axis=1)
df_fix_missing_values.head(2)

,funded_amount,loan_amount,activity,sector,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval,male_borrower_count,female_borrower_count
0,300.0,300.0,Fruits & Vegetables,Food,PK,Pakistan,Lahore,PKR,12.0,12,female,irregular,0.0,1.0
1,575.0,575.0,Rickshaw,Transportation,PK,Pakistan,Lahore,PKR,11.0,14,"female, female",irregular,0.0,2.0


**Fill column 'country_code'**

In [41]:
# Namibia is the only country that is missing country_codes
df_fix_missing_values.loc[df_fix_missing_values.country_code.isna()==True,'country_code':'country']

,country_code,country
198207,NaN,Namibia
198484,NaN,Namibia
337295,NaN,Namibia
343434,NaN,Namibia
411205,NaN,Namibia
411463,NaN,Namibia
473973,NaN,Namibia
474400,NaN,Namibia


In [42]:
# Tried to extract the country_code of Namibia via a non-missing row in the dataset, however all Namibia entries
# have a np.nan value. 
df_fix_missing_values.loc[df_fix_missing_values.country=='Namibia','country_code':'country']

,country_code,country
198207,NaN,Namibia
198484,NaN,Namibia
337295,NaN,Namibia
343434,NaN,Namibia
411205,NaN,Namibia
411463,NaN,Namibia
473973,NaN,Namibia
474400,NaN,Namibia


In [43]:
# Instead I had to source the ISO Alpha 2 code from the internet
# Source: https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes
df_fix_missing_values['country_code'].replace(to_replace=np.nan, value='NA', inplace=True)
df_fix_missing_values.loc[df_fix_missing_values.country_code.isna()==True,'country_code':'country']

,country_code,country


**Fill column 'region'**

In [44]:
for country in df_fix_missing_values.loc[:, 'country'].unique():
    mode_region = df_fix_missing_values.loc[df_fix_missing_values.country == country, 'region'].mode()
    if mode_region.empty: # Per Guam example, need to catch conditions where a country has no region entry across dataset
        df_fix_missing_values.loc[(df_fix_missing_values.country == country) & (df_fix_missing_values.region.isnull()), 'region'] = country
    else: # replace with most common region in this country
        df_fix_missing_values.loc[(df_fix_missing_values.country == country) & (df_fix_missing_values.region.isnull()), 'region'] = mode_region[0]

**Fill column 'borrower_genders'**

In [45]:
df_fix_missing_values['male_borrower_count'].describe()

count    641737.000000
mean          0.415729
std           1.136224
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max          44.000000
Name: male_borrower_count, dtype: float64

I use the median value to replace missing values due to the non-normal distribution of data in addition to extreme values.

In [46]:
male_borrower_median = df_fix_missing_values['male_borrower_count'].median()
df_fix_missing_values['male_borrower_count'].replace(np.nan, male_borrower_median, inplace=True)

In [47]:
df_fix_missing_values['female_borrower_count'].describe()

count    641737.000000
mean          1.613726
std           3.088613
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max          50.000000
Name: female_borrower_count, dtype: float64

Again, I use the median value to replace missing values due to the non-normal distribution of data in addition to extreme values.

In [48]:
female_borrower_median = df_fix_missing_values['female_borrower_count'].median()
df_fix_missing_values['female_borrower_count'].replace(np.nan, female_borrower_median, inplace=True)

In [49]:
# The total borrow count is then calculated for all rows by summing male and female borrower counts.
df_fix_missing_values["total_borrower_count"] = (
    df_fix_missing_values["male_borrower_count"]
    + df_fix_missing_values["female_borrower_count"]
)

df_fix_missing_values

,funded_amount,loan_amount,activity,sector,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval,male_borrower_count,female_borrower_count,total_borrower_count
0,300.0,300.0,Fruits & Vegetables,Food,PK,Pakistan,Lahore,PKR,12.0,12,female,irregular,0.0,1.0,1.0
1,575.0,575.0,Rickshaw,Transportation,PK,Pakistan,Lahore,PKR,11.0,14,"female, female",irregular,0.0,2.0,2.0
2,150.0,150.0,Transportation,Transportation,IN,India,Maynaguri,INR,43.0,6,female,bullet,0.0,1.0,1.0
3,200.0,200.0,Embroidery,Arts,PK,Pakistan,Lahore,PKR,11.0,8,female,irregular,0.0,1.0,1.0
4,400.0,400.0,Milk Sales,Food,PK,Pakistan,Abdul hakeem,PKR,14.0,16,female,monthly,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645620,0.0,25.0,Livestock,Agriculture,PY,Paraguay,Concepción,USD,13.0,0,female,monthly,0.0,1.0,1.0
645621,0.0,25.0,Livestock,Agriculture,PK,Pakistan,Attock,PKR,13.0,0,female,monthly,0.0,1.0,1.0
645622,0.0,125.0,Livestock,Agriculture,MX,Mexico,Iztacalco,MXN,13.0,0,"female, female",monthly,0.0,2.0,2.0
645623,0.0,875.0,Livestock,Agriculture,BO,Bolivia,La paz,BOB,13.0,0,"female, female",monthly,0.0,2.0,2.0


In [50]:
# We no longer need the borrower_genders column as the informationed is retained in other columns.
# We drop the column to reduce the memory footprint. 
df_fix_missing_values.drop('borrower_genders', axis=1, inplace=True)

Final check that all missing values have been handled

In [51]:
df_fix_missing_values.isnull().mean()*100

funded_amount            0.0
loan_amount              0.0
activity                 0.0
sector                   0.0
country_code             0.0
country                  0.0
region                   0.0
currency                 0.0
term_in_months           0.0
lender_count             0.0
repayment_interval       0.0
male_borrower_count      0.0
female_borrower_count    0.0
total_borrower_count     0.0
dtype: float64

### Outliers / Extreme Values


Here it must be checked whether the data lie within the framework in which the company moves.

**funded_amount**

The funded amount must be less than or equal to the loan amount. This was checked during the data quality checks and cases were caught, handled and passed to the relevant teams.

**loan_amount**

Within the business model there is no explicit upper limit to how much a borrower can request. There is a minimum limit of 25 USD, and this is the minimum value we see in this column.

In [52]:
df_fix_missing_values.loc[:,['loan_amount']].describe()

,loan_amount
count,645625.000000
mean,867.463427
std,1214.922030
min,25.000000
25%,275.000000
50%,500.000000
75%,1000.000000
max,100000.000000


**term_in_months**

Within the business model there is no explicit upper limit. The minimum value of one fits, as a time interval is required to pay the money out within.

In [53]:
df_fix_missing_values.loc[:,['term_in_months']].describe()

,term_in_months
count,645625.000000
mean,13.721730
std,8.477884
min,1.000000
25%,8.000000
50%,13.000000
75%,14.000000
max,158.000000


**lender_count**

Within the business model there is no explicit upper limit for how many investors can be involved in funding a loan. The minimum lender should not be lower than zero as this would be a business bug. All values are within this range.

In [54]:
df_fix_missing_values.loc[:,['lender_count']].describe()

,lender_count
count,645625.000000
mean,21.136911
std,28.856818
min,0.000000
25%,7.000000
50%,13.000000
75%,25.000000
max,2986.000000


**x_borrower_count columns**

Within the business model there is no explicit upper limit for how many people are associated with a loan. At least one borrower is required to be associated with a loan, which is fulfilled across all loans. 

In [55]:
df_fix_missing_values.loc[:,'male_borrower_count':'total_borrower_count'].describe()

,male_borrower_count,female_borrower_count,total_borrower_count
count,645625.000000,645625.000000,645625.000000
mean,0.413226,1.610031,2.023257
std,1.133255,3.079665,3.461317
min,0.000000,0.000000,1.000000
25%,0.000000,1.000000,1.000000
50%,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000
max,44.000000,50.000000,50.000000


### KPIs / Features

In [56]:
df_kpis = df_fix_missing_values

# %_loan_funded = funded_amount / loan_amount 
df_kpis['%_loan_funded'] = df_kpis['funded_amount']/df_kpis['loan_amount']

# funding per lender = funding_amount / lender_count 
df_kpis['funding_per_lender'] = df_kpis['funded_amount']/df_kpis['lender_count']

# loan_term_per_month = loan_amount / term_in_months 
df_kpis['loan_term_per_month'] = df_kpis['loan_amount']/df_kpis['term_in_months']

# fund_term_per_month = funded_amount / term_in months 
df_kpis['fund_term_per_month'] = df_kpis['funded_amount']/df_kpis['term_in_months']

# loan_per_borrower = loan_amount / total_borrower_count 
df_kpis['loan_per_borrower'] = df_kpis['loan_amount']/df_kpis['total_borrower_count']

# fund_per_borrower = funded_amount / total_borrower_count 
df_kpis['fund_per_borrower'] = df_kpis['funded_amount']/df_kpis['total_borrower_count']

# m_borrower_% = male_borrower_count / total_borrower_count 
df_kpis['m_borrower_%'] = df_kpis['male_borrower_count']/df_kpis['total_borrower_count']

# f_borrower_% = female_borrower_count / total_borrower_count 
df_kpis['f_borrower_%'] = df_kpis['female_borrower_count']/df_kpis['total_borrower_count']

In [57]:
df_kpis.head(2)

,funded_amount,loan_amount,activity,sector,country_code,country,region,currency,term_in_months,lender_count,...,female_borrower_count,total_borrower_count,%_loan_funded,funding_per_lender,loan_term_per_month,fund_term_per_month,loan_per_borrower,fund_per_borrower,m_borrower_%,f_borrower_%
0,300.0,300.0,Fruits & Vegetables,Food,PK,Pakistan,Lahore,PKR,12.0,12,...,1.0,1.0,1.0,25.000000,25.000000,25.000000,300.0,300.0,0.0,1.0
1,575.0,575.0,Rickshaw,Transportation,PK,Pakistan,Lahore,PKR,11.0,14,...,2.0,2.0,1.0,41.071429,52.272727,52.272727,287.5,287.5,0.0,1.0


In [58]:
# funding_per_lender returns NaN when we try to divide by zero. Since there are no lenders, updating values to zero
df_kpis.isnull().mean()*100

funded_amount            0.000000
loan_amount              0.000000
activity                 0.000000
sector                   0.000000
country_code             0.000000
country                  0.000000
region                   0.000000
currency                 0.000000
term_in_months           0.000000
lender_count             0.000000
repayment_interval       0.000000
male_borrower_count      0.000000
female_borrower_count    0.000000
total_borrower_count     0.000000
%_loan_funded            0.000000
funding_per_lender       0.507106
loan_term_per_month      0.000000
fund_term_per_month      0.000000
loan_per_borrower        0.000000
fund_per_borrower        0.000000
m_borrower_%             0.000000
f_borrower_%             0.000000
dtype: float64

In [59]:
df_kpis['funding_per_lender'].replace(np.nan, 0, inplace=True)
df_kpis['funding_per_lender'].isnull().mean()*100

0.0

In [60]:
# Final check that all missing values have been handled
df_kpis.isnull().mean()*100

funded_amount            0.0
loan_amount              0.0
activity                 0.0
sector                   0.0
country_code             0.0
country                  0.0
region                   0.0
currency                 0.0
term_in_months           0.0
lender_count             0.0
repayment_interval       0.0
male_borrower_count      0.0
female_borrower_count    0.0
total_borrower_count     0.0
%_loan_funded            0.0
funding_per_lender       0.0
loan_term_per_month      0.0
fund_term_per_month      0.0
loan_per_borrower        0.0
fund_per_borrower        0.0
m_borrower_%             0.0
f_borrower_%             0.0
dtype: float64

### Datatype conversions for space efficiency

In [61]:
# memory foot before converting datatypes
memory_before = df_kpis.memory_usage(deep=True).sum()
memory_before

371165988

In [62]:
# Check whether all floats in funded_amount are round values
df_kpis.funded_amount.apply(float.is_integer).all()

True

In [63]:
# Check whether all floats in loan_amount are round values
df_kpis.loan_amount.apply(float.is_integer).all()

True

In [64]:
# Check whether all floats in term_in_months are round values
df_kpis.term_in_months.apply(float.is_integer).all()

True

In [65]:
# Float columns that only use whole numbers can be converted into datatype int to save space and not lose information
# Object columns that do not contain unique values across all rows can be coverted to datatype category to save space

df_kpis = df_kpis.astype(
    {'activity':'category', 
     'sector':'category',
     'country_code':'category', 
     'country':'category',
     'region':'category', 
     'currency':'category',
     'repayment_interval':'category',
     'funded_amount':'int',
     'loan_amount':'int',
     'term_in_months':'int',
     'male_borrower_count':'int',
     'female_borrower_count':'int',
     'total_borrower_count':'int',
     'lender_count':'int',
    }
)

In [66]:
df_kpis.dtypes

funded_amount               int32
loan_amount                 int32
activity                 category
sector                   category
country_code             category
country                  category
region                   category
currency                 category
term_in_months              int32
lender_count                int32
repayment_interval       category
male_borrower_count         int32
female_borrower_count       int32
total_borrower_count        int32
%_loan_funded             float64
funding_per_lender        float64
loan_term_per_month       float64
fund_term_per_month       float64
loan_per_borrower         float64
fund_per_borrower         float64
m_borrower_%              float64
f_borrower_%              float64
dtype: object

In [67]:
memory_after = df_kpis.memory_usage(deep=True).sum()
memory_saved = (1 - (memory_after / memory_before)) * 100
print(f'Memory foot print before: {memory_before:,}')
print(f'Memory foot print after: {memory_after:,}')
print(f'Total memory foot print reduction: {memory_before - memory_after:,}')
print(f'Memory foot print reduction %: {memory_saved:.2f}%')

Memory foot print before: 371,165,988
Memory foot print after: 66,814,209
Total memory foot print reduction: 304,351,779
Memory foot print reduction %: 82.00%


### Save dataframe for explorative data analysis

In [69]:
df_kpis.to_pickle('data_post_processing.pkl')